In [26]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_dir = '/home/jihyeparkk/GIT/KcBERT-Finetune/ckpt/kcbert-base-nsmc-ckpt/checkpoint-1000'

In [25]:
tokenizer = AutoTokenizer.from_pretrained('beomi/kcbert-base')
tokenizer.save_pretrained(model_dir)

('/home/jihyeparkk/GIT/KcBERT-Finetune/ckpt/kcbert-base-nsmc-ckpt/checkpoint-1000/tokenizer_config.json',
 '/home/jihyeparkk/GIT/KcBERT-Finetune/ckpt/kcbert-base-nsmc-ckpt/checkpoint-1000/special_tokens_map.json',
 '/home/jihyeparkk/GIT/KcBERT-Finetune/ckpt/kcbert-base-nsmc-ckpt/checkpoint-1000/vocab.txt',
 '/home/jihyeparkk/GIT/KcBERT-Finetune/ckpt/kcbert-base-nsmc-ckpt/checkpoint-1000/added_tokens.json',
 '/home/jihyeparkk/GIT/KcBERT-Finetune/ckpt/kcbert-base-nsmc-ckpt/checkpoint-1000/tokenizer.json')

In [28]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [29]:
model = AutoModelForSequenceClassification.from_pretrained(model_dir)
model = model.to(device)
tokenizer = AutoTokenizer.from_pretrained(model_dir)

In [36]:
import torch

def inference(model, input_ids, attention_mask):
    model.eval()
    with torch.no_grad():
        try: outputs = model(input_ids, attention_mask=attention_mask, return_dict=True)
        except: return None
    logits = outputs['logits']
    predicted_label = torch.argmax(logits)
    return predicted_label.item()

def encode_for_inference(device, tokenizer, text):
    data = {key: torch.tensor(val) for key, val in tokenizer(text, truncation=True, padding='max_length', max_length=512).items()}
    return torch.unsqueeze(data['input_ids'], 0).to(device), torch.unsqueeze(data['attention_mask'], 0).to(device)

In [44]:
texts = ["이 마스크는 저렴해서 좋아요", "이 마스크는 예뻐서 좋아요", "이 마스크는 크기가 커서 좋아요"]
for text in texts:
    input_ids, attention_mask = encode_for_inference(device, tokenizer, text)
    print(text)
    print(inference(model, input_ids, attention_mask), '\n')

이 마스크는 저렴해서 좋아요
0 

이 마스크는 예뻐서 좋아요
1 

이 마스크는 크기가 커서 좋아요
2 



2